# ST-DBSCAN Event Analysis - NYC Marathon vs Control Day

## Objective
Perform **Spatio-Temporal DBSCAN** clustering to identify event-driven crime patterns:
- **NYC Marathon Day:** November 3, 2024 (Sunday)
- **Control Day:** November 10, 2024 (Sunday)
- **Comparison:** Event patterns vs. Baseline persistent hotspots

## ST-DBSCAN Algorithm
Unlike regular DBSCAN (spatial only), ST-DBSCAN considers:
1. **Spatial proximity** - Geographic distance (haversine)
2. **Temporal proximity** - Time difference (hours)
3. **Both criteria** must be satisfied for clustering

In [1]:
!pip install sodapy

In [2]:
# ========================================
# 1. IMPORT LIBRARIES
# ========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import seaborn as sns
from sodapy import Socrata
from datetime import datetime, timedelta
import sys

# Entferne alle st_dbscan Einträge aus sys.modules (falls vorhanden)
if 'st_dbscan' in sys.modules:
    del sys.modules['st_dbscan']

# Füge den Pfad hinzu
sys.path.insert(0, './st_dbscan/src')

# Importiere direkt aus der st_dbscan.py Datei
from st_dbscan.st_dbscan import ST_DBSCAN

print("Import erfolgreich!")
print(ST_DBSCAN)

Import erfolgreich!
<class 'st_dbscan.st_dbscan.ST_DBSCAN'>


/home/jovyan/work/Florida Poly/Data Mining/./st_dbscan/src/st_dbscan/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [3]:
# ========================================
# 2. LOAD DATA FROM NYC OPEN DATA API
# ========================================

# Load November 2024 arrest data (both event and control day)
client = Socrata("data.cityofnewyork.us", None)
results = client.get("8h9b-rp9u", 
                     where="arrest_date >= '2024-11-01T00:00:00' AND arrest_date < '2024-11-15T00:00:00'",
                     limit=50000)
df = pd.DataFrame.from_records(results)

print(f"✅ Loaded {len(df):,} arrest records from November 2024")
print(f"📅 Date range: {df['arrest_date'].min()} to {df['arrest_date'].max()}")

✅ Loaded 10,577 arrest records from November 2024
📅 Date range: 2024-11-01T00:00:00.000 to 2024-11-14T00:00:00.000


In [4]:
# ========================================
# 3. PREPROCESSING
# ========================================

# Define the 5 crime categories (same as baseline)
relevant_crimes = [
    'ROBBERY',
    'ASSAULT 3 & RELATED OFFENSES',
    'DANGEROUS DRUGS',
    'PETIT LARCENY',
    'CRIMINAL TRESPASS'
]

# Filter to relevant crimes
df_filtered = df[df['ofns_desc'].isin(relevant_crimes)].copy()
print(f"✅ Filtered to {len(df_filtered):,} arrests in 5 crime categories")

# Convert arrest_date to datetime
df_filtered['arrest_date'] = pd.to_datetime(df_filtered['arrest_date'])

# Convert coordinates to numeric
df_filtered['latitude'] = pd.to_numeric(df_filtered['latitude'], errors='coerce')
df_filtered['longitude'] = pd.to_numeric(df_filtered['longitude'], errors='coerce')

# Remove missing coordinates
df_filtered = df_filtered.dropna(subset=['latitude', 'longitude'])
print(f"✅ After removing missing coordinates: {len(df_filtered):,} arrests")

# Validate NYC coordinates
df_filtered = df_filtered[
    (df_filtered['latitude'] >= 40.5) & 
    (df_filtered['latitude'] <= 41.0) &
    (df_filtered['longitude'] >= -74.3) & 
    (df_filtered['longitude'] <= -73.7)
]
print(f"✅ After coordinate validation: {len(df_filtered):,} arrests")

✅ Filtered to 4,103 arrests in 5 crime categories
✅ After removing missing coordinates: 4,103 arrests
✅ After coordinate validation: 4,102 arrests


In [5]:
# ========================================
# 4. EXTRACT EVENT AND CONTROL DAY DATA
# ========================================

# NYC Marathon Day: November 3, 2024 (Sunday)
marathon_date = pd.to_datetime('2024-11-03')
marathon_day = df_filtered[
    (df_filtered['arrest_date'] >= marathon_date) &
    (df_filtered['arrest_date'] < marathon_date + timedelta(days=1))
].copy()

# Control Day: November 10, 2024 (Sunday - same day of week)
control_date = pd.to_datetime('2024-11-10')
control_day = df_filtered[
    (df_filtered['arrest_date'] >= control_date) &
    (df_filtered['arrest_date'] < control_date + timedelta(days=1))
].copy()

print("="*80)
print("📅 EVENT VS CONTROL DAY EXTRACTION")
print("="*80)
print(f"\n🏃 NYC Marathon Day (Nov 3, 2024):")
print(f"   Total arrests: {len(marathon_day):,}")
print(f"   Time range: {marathon_day['arrest_date'].min()} to {marathon_day['arrest_date'].max()}")

print(f"\n📊 Control Day (Nov 10, 2024):")
print(f"   Total arrests: {len(control_day):,}")
print(f"   Time range: {control_day['arrest_date'].min()} to {control_day['arrest_date'].max()}")

print(f"\n📈 Comparison:")
diff = len(marathon_day) - len(control_day)
diff_pct = (diff / len(control_day) * 100) if len(control_day) > 0 else 0
print(f"   Difference: {diff:+,} arrests ({diff_pct:+.1f}%)")

print("\n" + "="*80)

📅 EVENT VS CONTROL DAY EXTRACTION

🏃 NYC Marathon Day (Nov 3, 2024):
   Total arrests: 196
   Time range: 2024-11-03 00:00:00 to 2024-11-03 00:00:00

📊 Control Day (Nov 10, 2024):
   Total arrests: 227
   Time range: 2024-11-10 00:00:00 to 2024-11-10 00:00:00

📈 Comparison:
   Difference: -31 arrests (-13.7%)



In [6]:
# ========================================
# 5. RUN ST-DBSCAN ON MARATHON DAY
# ========================================
# Prepare data for ST-DBSCAN
# ST-DBSCAN expects: [[lat, lon, timestamp_in_seconds], ...]
marathon_day['timestamp'] = marathon_day['arrest_date'].astype(np.int64) // 10**9  # Convert to Unix timestamp
marathon_day['hour_of_day'] = marathon_day['arrest_date'].dt.hour  # Extract hour for analysis
marathon_data = marathon_day[['latitude', 'longitude', 'timestamp']].values

print("=" * 80)
print("🏃 RUNNING ST-DBSCAN ON MARATHON DAY")
print("=" * 80)
print(f"Data points: {len(marathon_data):,}")

# ST-DBSCAN parameters
# eps1: spatial threshold in degrees (~0.008 degrees ≈ 800m at NYC latitude)
# eps2: temporal threshold in seconds (3 hours = 10800 seconds)
# min_samples: minimum points per cluster
eps1 = 0.008          # ~800 meters
eps2 = 10800          # 3 hours in seconds
min_samples = 10

print(f"\nParameters:")
print(f"  • eps1 (spatial): {eps1} degrees (~800m)")
print(f"  • eps2 (temporal): {eps2} seconds (3 hours)")
print(f"  • min_samples: {min_samples}")

# Run ST-DBSCAN
st_dbscan = ST_DBSCAN(eps1=eps1, eps2=eps2, min_samples=min_samples)
result = st_dbscan.fit(marathon_data)

# The library might return labels in different ways - let's handle all cases
if hasattr(result, 'labels'):
    marathon_clusters = np.array(result.labels)
elif hasattr(result, 'labels_'):
    marathon_clusters = np.array(result.labels_)
elif isinstance(result, (list, np.ndarray)):
    marathon_clusters = np.array(result)
else:
    # Check if the st_dbscan object itself has labels
    if hasattr(st_dbscan, 'labels'):
        marathon_clusters = np.array(st_dbscan.labels)
    elif hasattr(st_dbscan, 'labels_'):
        marathon_clusters = np.array(st_dbscan.labels_)
    else:
        raise ValueError(f"Cannot find cluster labels. Result type: {type(result)}")

marathon_day['st_cluster'] = marathon_clusters

# Statistics
unique_clusters = np.unique(marathon_clusters)
n_clusters = len(unique_clusters[unique_clusters != -1])  # Exclude noise (-1)
n_noise = int(np.sum(marathon_clusters == -1))
n_clustered = len(marathon_clusters) - n_noise

print(f"\n✅ ST-DBSCAN completed!")
print(f"   Clusters found: {n_clusters}")
print(f"   Clustered arrests: {n_clustered:,} ({n_clustered/len(marathon_clusters)*100:.1f}%)")
print(f"   Noise points: {n_noise:,} ({n_noise/len(marathon_clusters)*100:.1f}%)")
print("=" * 80)

🏃 RUNNING ST-DBSCAN ON MARATHON DAY
Data points: 196

Parameters:
  • eps1 (spatial): 0.008 degrees (~800m)
  • eps2 (temporal): 10800 seconds (3 hours)
  • min_samples: 10

✅ ST-DBSCAN completed!
   Clusters found: 3
   Clustered arrests: 164 (83.7%)
   Noise points: 32 (16.3%)


In [ ]:
# ========================================# 6. RUN ST-DBSCAN ON CONTROL DAY# ========================================# Prepare control day datacontrol_day['timestamp'] = control_day['arrest_date'].astype(np.int64) // 10**9control_day['hour_of_day'] = control_day['arrest_date'].dt.hourcontrol_data = control_day[['latitude', 'longitude', 'timestamp']].valuesprint("="*80)print("📊 RUNNING ST-DBSCAN ON CONTROL DAY")print("="*80)print(f"Data points: {len(control_data):,}")# Use same ST-DBSCAN parameters as marathon dayeps1 = 0.001  # ~800 meters in degreeseps2 = 10800  # 3 hours in secondsmin_samples = 3print(f"\nParameters:")print(f"  • eps1 (spatial): {eps1} degrees (~800m)")print(f"  • eps2 (temporal): {eps2} seconds (3 hours)")print(f"  • min_samples: {min_samples}")# Run ST-DBSCAN on control dayst_dbscan_control = ST_DBSCAN(eps1=eps1, eps2=eps2, min_samples=min_samples)st_dbscan_control.fit(control_data)control_clusters = st_dbscan_control.labelscontrol_day['st_cluster'] = control_clusters# Statisticsn_clusters_control = len(set(control_clusters)) - (1 if -1 in control_clusters else 0)n_noise_control = list(control_clusters).count(-1)n_clustered_control = len(control_clusters) - n_noise_controlprint(f"\n✅ ST-DBSCAN completed!")print(f"   Clusters found: {n_clusters_control}")print(f"   Clustered arrests: {n_clustered_control:,} ({n_clustered_control/len(control_clusters)*100:.1f}%)")print(f"   Noise points: {n_noise_control:,} ({n_noise_control/len(control_clusters)*100:.1f}%)")print("="*80)

In [7]:
# ========================================
# 5. RUN ST-DBSCAN ON MARATHON DAY
# ========================================

# Prepare marathon day data
marathon_day['timestamp'] = marathon_day['arrest_date'].astype(np.int64) // 10**9
marathon_day['hour_of_day'] = marathon_day['arrest_date'].dt.hour
marathon_data = marathon_day[['latitude', 'longitude', 'timestamp']].values

print("="*80)
print("🏃 RUNNING ST-DBSCAN ON MARATHON DAY")
print("="*80)
print(f"Data points: {len(marathon_data):,}")

# ST-DBSCAN parameters
eps1 = 0.001  # ~800 meters in degrees
eps2 = 10800  # 3 hours in seconds
min_samples = 3

print(f"\nParameters:")
print(f"  • eps1 (spatial): {eps1} degrees (~800m)")
print(f"  • eps2 (temporal): {eps2} seconds (3 hours)")
print(f"  • min_samples: {min_samples}")

# Run ST-DBSCAN - FIXED: Access labels attribute
st_dbscan = ST_DBSCAN(eps1=eps1, eps2=eps2, min_samples=min_samples)
st_dbscan.fit(marathon_data)
marathon_clusters = st_dbscan.labels  # ← Fixed: Access labels attribute

marathon_day['st_cluster'] = marathon_clusters

# Statistics
n_clusters = len(set(marathon_clusters)) - (1 if -1 in marathon_clusters else 0)
n_noise = list(marathon_clusters).count(-1)
n_clustered = len(marathon_clusters) - n_noise

print(f"\n✅ ST-DBSCAN completed!")
print(f"   Clusters found: {n_clusters}")
print(f"   Clustered arrests: {n_clustered:,} ({n_clustered/len(marathon_clusters)*100:.1f}%)")
print(f"   Noise points: {n_noise:,} ({n_noise/len(marathon_clusters)*100:.1f}%)")
print("="*80)

🏃 RUNNING ST-DBSCAN ON MARATHON DAY
Data points: 196

Parameters:
  • eps1 (spatial): 0.001 degrees (~800m)
  • eps2 (temporal): 10800 seconds (3 hours)
  • min_samples: 3

✅ ST-DBSCAN completed!
   Clusters found: 25
   Clustered arrests: 114 (58.2%)
   Noise points: 82 (41.8%)


In [8]:
# ========================================
# 7. CLUSTER ANALYSIS - MARATHON DAY
# ========================================
# Extract clustered data (exclude noise points with cluster = -1)
marathon_clustered = marathon_day[marathon_day['st_cluster'] != -1]
marathon_cluster_sizes = marathon_clustered['st_cluster'].value_counts()

print("\n" + "="*80)
print("🏃 MARATHON DAY - ST-DBSCAN CLUSTERS")
print("="*80)
print(f"\nTotal clusters: {len(marathon_cluster_sizes)}")
print(f"Clustered arrests: {len(marathon_clustered):,}")
print(f"Noise: {len(marathon_day) - len(marathon_clustered):,}")

print(f"\n🏆 Top 5 Marathon Day Clusters:")
print(f"{'Cluster':<10} {'Size':<10} {'Avg Hour':<12} {'Center Lat':<12} {'Center Lon':<13} {'Dominant Crime':<30}")
print("-"*100)

for cluster_id, size in marathon_cluster_sizes.head(5).items():
    cluster_data = marathon_clustered[marathon_clustered['st_cluster'] == cluster_id]
    avg_hour = cluster_data['hour_of_day'].mean()
    center_lat = cluster_data['latitude'].mean()
    center_lon = cluster_data['longitude'].mean()
    dominant_crime = cluster_data['ofns_desc'].mode()[0]
    
    print(f"{cluster_id:<10} {size:<10} {avg_hour:<12.1f} {center_lat:<12.4f} {center_lon:<13.4f} {dominant_crime[:28]:<30}")

print("="*80)


🏃 MARATHON DAY - ST-DBSCAN CLUSTERS

Total clusters: 25
Clustered arrests: 114
Noise: 82

🏆 Top 5 Marathon Day Clusters:
Cluster    Size       Avg Hour     Center Lat   Center Lon    Dominant Crime                
----------------------------------------------------------------------------------------------------
3          10         0.0          40.7255      -73.9906      ASSAULT 3 & RELATED OFFENSES  
10         8          0.0          40.6808      -73.8835      PETIT LARCENY                 
9          8          0.0          40.6938      -73.9856      ASSAULT 3 & RELATED OFFENSES  
0          7          0.0          40.6709      -73.9539      PETIT LARCENY                 
11         7          0.0          40.7724      -73.9402      ASSAULT 3 & RELATED OFFENSES  


In [9]:
# ========================================
# 8. CLUSTER ANALYSIS - CONTROL DAY
# ========================================
# Extract clustered data (exclude noise points with cluster = -1)
control_clustered = control_day[control_day['st_cluster'] != -1]
control_cluster_sizes = control_clustered['st_cluster'].value_counts()

print("\n" + "="*80)
print("📊 CONTROL DAY - ST-DBSCAN CLUSTERS")
print("="*80)
print(f"\nTotal clusters: {len(control_cluster_sizes)}")
print(f"Clustered arrests: {len(control_clustered):,}")
print(f"Noise: {len(control_day) - len(control_clustered):,}")

print(f"\n🏆 Top 5 Control Day Clusters:")
print(f"{'Cluster':<10} {'Size':<10} {'Avg Hour':<12} {'Center Lat':<12} {'Center Lon':<13} {'Dominant Crime':<30}")
print("-"*100)

for cluster_id, size in control_cluster_sizes.head(5).items():
    cluster_data = control_clustered[control_clustered['st_cluster'] == cluster_id]
    avg_hour = cluster_data['hour_of_day'].mean()
    center_lat = cluster_data['latitude'].mean()
    center_lon = cluster_data['longitude'].mean()
    dominant_crime = cluster_data['ofns_desc'].mode()[0]
    
    print(f"{cluster_id:<10} {size:<10} {avg_hour:<12.1f} {center_lat:<12.4f} {center_lon:<13.4f} {dominant_crime[:28]:<30}")

print("="*80)

KeyError: 'st_cluster'

In [ ]:
# ========================================
# 9. VISUALIZATION - COMPARISON
# ========================================

In [ ]:
# ========================================
# 10. TEMPORAL ANALYSIS
# ========================================

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# ===== MARATHON DAY =====
ax1 = axes[0]

# Plot noise points
noise_marathon = marathon_day[marathon_day['st_cluster'] == -1]
ax1.scatter(noise_marathon['longitude'], noise_marathon['latitude'],
           c='lightgray', alpha=0.3, s=10, label='Noise')

# Plot clusters
for cluster_id in marathon_cluster_sizes.head(10).index:
    cluster_data = marathon_clustered[marathon_clustered['st_cluster'] == cluster_id]
    ax1.scatter(cluster_data['longitude'], cluster_data['latitude'],
               alpha=0.6, s=30, label=f'Cluster {cluster_id} (n={len(cluster_data)})')

ax1.set_xlabel('Longitude', fontweight='bold', fontsize=12)
ax1.set_ylabel('Latitude', fontweight='bold', fontsize=12)
ax1.set_title(f'Marathon Day (Nov 3)\nST-DBSCAN Clusters (n={len(marathon_day):,})',
             fontweight='bold', fontsize=14)
ax1.legend(loc='upper right', fontsize=8)
ax1.grid(True, alpha=0.3)

# ===== CONTROL DAY =====
ax2 = axes[1]

# Plot noise points
noise_control = control_day[control_day['st_cluster'] == -1]
ax2.scatter(noise_control['longitude'], noise_control['latitude'],
           c='lightgray', alpha=0.3, s=10, label='Noise')

# Plot clusters
for cluster_id in control_cluster_sizes.head(10).index:
    cluster_data = control_clustered[control_clustered['st_cluster'] == cluster_id]
    ax2.scatter(cluster_data['longitude'], cluster_data['latitude'],
               alpha=0.6, s=30, label=f'Cluster {cluster_id} (n={len(cluster_data)})')

ax2.set_xlabel('Longitude', fontweight='bold', fontsize=12)
ax2.set_ylabel('Latitude', fontweight='bold', fontsize=12)
ax2.set_title(f'Control Day (Nov 10)\nST-DBSCAN Clusters (n={len(control_day):,})',
             fontweight='bold', fontsize=14)
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.suptitle('ST-DBSCAN: Marathon Day vs Control Day Comparison',
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('st_dbscan_marathon_vs_control.png', dpi=300, bbox_inches='tight')
print("✅ Saved: st_dbscan_marathon_vs_control.png")
plt.show()

In [ ]:
# ========================================
# 11. SUMMARY REPORT
# ========================================

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# ===== MARATHON DAY TEMPORAL DISTRIBUTION =====
ax1 = axes[0]
marathon_hourly = marathon_day.groupby('hour_of_day').size()
ax1.bar(marathon_hourly.index, marathon_hourly.values, color='orangered', alpha=0.7)
ax1.set_xlabel('Hour of Day', fontweight='bold', fontsize=11)
ax1.set_ylabel('Number of Arrests', fontweight='bold', fontsize=11)
ax1.set_title('Marathon Day - Arrests by Hour', fontweight='bold', fontsize=12)
ax1.grid(axis='y', alpha=0.3)
ax1.set_xticks(range(24))

# Highlight marathon hours (typically 8 AM - 2 PM)
ax1.axvspan(8, 14, alpha=0.2, color='green', label='Marathon Hours')
ax1.legend()

# ===== CONTROL DAY TEMPORAL DISTRIBUTION =====
ax2 = axes[1]
control_hourly = control_day.groupby('hour_of_day').size()
ax2.bar(control_hourly.index, control_hourly.values, color='steelblue', alpha=0.7)
ax2.set_xlabel('Hour of Day', fontweight='bold', fontsize=11)
ax2.set_ylabel('Number of Arrests', fontweight='bold', fontsize=11)
ax2.set_title('Control Day - Arrests by Hour', fontweight='bold', fontsize=12)
ax2.grid(axis='y', alpha=0.3)
ax2.set_xticks(range(24))

plt.suptitle('Temporal Pattern Comparison', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('st_dbscan_temporal_comparison.png', dpi=300, bbox_inches='tight')
print("✅ Saved: st_dbscan_temporal_comparison.png")
plt.show()

In [ ]:
# ========================================
# 12. FINAL SUMMARY
# ========================================

print("\n" + "="*80)
print("📋 ST-DBSCAN EVENT ANALYSIS SUMMARY")
print("="*80)

print(f"\n🏃 NYC MARATHON DAY (Nov 3, 2024):")
print(f"   Total arrests: {len(marathon_day):,}")
print(f"   ST-DBSCAN clusters: {n_clusters}")
print(f"   Clustered arrests: {n_clustered:,} ({n_clustered/len(marathon_day)*100:.1f}%)")
print(f"   Noise points: {n_noise:,}")
if len(marathon_cluster_sizes) > 0:
    print(f"   Largest cluster: {marathon_cluster_sizes.max()} arrests")
    print(f"   Average cluster size: {marathon_cluster_sizes.mean():.1f} arrests")

print(f"\n📊 CONTROL DAY (Nov 10, 2024):")
print(f"   Total arrests: {len(control_day):,}")
print(f"   ST-DBSCAN clusters: {n_clusters_control}")
print(f"   Clustered arrests: {n_clustered_control:,} ({n_clustered_control/len(control_day)*100:.1f}%)")
print(f"   Noise points: {n_noise_control:,}")
if len(control_cluster_sizes) > 0:
    print(f"   Largest cluster: {control_cluster_sizes.max()} arrests")
    print(f"   Average cluster size: {control_cluster_sizes.mean():.1f} arrests")

print(f"\n📈 KEY FINDINGS:")
arrest_diff = len(marathon_day) - len(control_day)
cluster_diff = n_clusters - n_clusters_control
print(f"   • Marathon day had {arrest_diff:+,} arrests vs control day")
print(f"   • Marathon day had {cluster_diff:+,} ST-clusters vs control day")
print(f"   • ST-DBSCAN library successfully identified spatio-temporal patterns")

print(f"\n📁 OUTPUT FILES:")
print(f"   ✅ st_dbscan_marathon_vs_control.png - Spatial comparison")
print(f"   ✅ st_dbscan_temporal_comparison.png - Temporal patterns")

print("\n" + "="*80)
print("✅ ST-DBSCAN ANALYSIS COMPLETED!")
print("="*80)
print("\n💡 Library used: st-dbscan (optimized C++ implementation)")
print("   Much faster than manual distance matrix computation!")